# OPUS Report Decoder.
> Decode NOAA NGS OPUS Reports into a Python class variable.

In [1]:
#| default_exp decode_OPUS

#|hide
### Test Data.

Test OPUS data file in a string.

In [2]:
opus_txt = '''
FILE: SFKW1210.24O OP1716177177447

 1008   NOTE: You provided a zero or negative antenna height.
 1008   If ARP HGT = 0.0, OPUS solves for the position of your selected antenna's reference point (ARP).
 1008   If ARP HGT < 0.0, OPUS solves for a location inside or above the antenna
 1008
                              NGS OPUS SOLUTION REPORT
                              ========================

All computed coordinate accuracies are listed as peak-to-peak values.
For additional information: https://www.ngs.noaa.gov/OPUS/about.jsp#accuracy

      USER: abcdef@xderxxxc.com                      DATE: May 20, 2024
RINEX FILE: sfkw121p.24o                             TIME: 03:53:37 UTC


  SOFTWARE: page5  2008.25 master253.pl 160321      START: 2024/04/30  15:51:00
 EPHEMERIS: igs23122.eph [precise]                   STOP: 2024/04/30  22:51:00
  NAV FILE: brdc1210.24n                         OBS USED: 14342 / 14604   :  98%
  ANT NAME: TRMR10-2        NONE              # FIXED AMB:    65 /    68   :  96%
ARP HEIGHT: 0.000                             OVERALL RMS: 0.014(m)


 REF FRAME: NAD_83(2011)(EPOCH:2010.0000)              ITRF2014 (EPOCH:2024.3301)

         X:     -2475273.328(m)   0.019(m)          -2475274.384(m)   0.019(m)
         Y:     -4509243.734(m)   0.013(m)          -4509242.286(m)   0.013(m)
         Z:      3760194.885(m)   0.015(m)           3760194.828(m)   0.015(m)

       LAT:   36 21  0.18896      0.022(m)        36 21  0.20210      0.022(m)
     E LON:  241 14 10.16711      0.012(m)       241 14 10.10206      0.012(m)
     W LON:  118 45 49.83289      0.012(m)       118 45 49.89794      0.012(m)
    EL HGT:         1094.647(m)   0.012(m)              1094.000(m)   0.012(m)
 ORTHO HGT:         1122.958(m)   0.068(m) [NAVD88 (Computed using GEOID18)]

                        UTM COORDINATES    STATE PLANE COORDINATES
                         UTM (Zone 11)         SPC (0404 CA 4)
Northing (Y) [meters]     4024219.772           612842.492
Easting (X)  [meters]      341727.091          2021197.302
Convergence  [degrees]    -1.04567778           0.14088889
Point Scale                0.99990865           0.99995228
Combined Factor            0.99973690           0.99978052

US NATIONAL GRID DESIGNATOR: 11SLA4172724220(NAD 83)


                              BASE STATIONS USED
PID       DESIGNATION                        LATITUDE    LONGITUDE DISTANCE(m)
DM6188 P572 SHADEQUARTCS2006 CORS GRP      N363507.843 W1185716.494   31229.8
DN7424 P567 RIOBRAVO_CS2005 CORS GRP       N352515.393 W1184512.826  103109.5
DK6413 P467 ALABAMAHILCS2006 CORS GRP      N363412.712 W1180526.193   65116.5

                 NEAREST NGS PUBLISHED CONTROL POINT
GT1835      DENNISON                       N361906.411 W1184447.601    3835.1

This position and the above vector components were computed without any
knowledge by the National Geodetic Survey regarding the equipment or
field operating procedures used.
'''

#|hide
## Code.

#|hide
### Imports

In [3]:
#| export
import datetime as dt
import numpy    as np
import time 

The following rows from an OPUS Report are decoded.
```
          1         2         3         4         5         6         7         8
0123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890
                              NGS OPUS SOLUTION REPORT
                              NGS OPUS-RS SOLUTION REPORT
      USER: abcdefgh@XXXXXXXXX                       DATE: May 20, 2024
RINEX FILE: sfkw121p.24o                             TIME: 03:53:37 UTC
  SOFTWARE: page5  2008.25 master240.pl 160321      START: 2024/05/30  00:00:00
  SOFTWARE: rsgps  1.38 RS272.prl 1.99.3            START: 2024/05/30 00:00:00
 EPHEMERIS: igs23122.eph [precise]                   STOP: 2024/04/30  22:51:00
  ANT NAME: TRMR10-2        NONE              # FIXED AMB:    65 /    68   :  96%
ARP HEIGHT: 0.000                             OVERALL RMS: 0.014(m)
 REF FRAME: NAD_83(2011)(EPOCH:2010.0000)              ITRF2014 (EPOCH:2024.3301)
         X:     -2475273.328(m)   0.019(m)          -2475274.384(m)   0.019(m)
         Y:     -4509243.734(m)   0.013(m)          -4509242.286(m)   0.013(m)
         Z:      3760194.885(m)   0.015(m)           3760194.828(m)   0.015(m)
       LAT:   37 56  3.46294      0.001(m)        37 56  3.49404      0.001(m)
     W LON:  118 45 49.83289      0.012(m)       118 45 49.89794      0.012(m)
    EL HGT:         1094.647(m)   0.012(m)              1094.000(m)   0.012(m)
 ORTHO HGT:           26.321(m)   0.061(m) [NAVD88 (Computed using GEOID18)]
 ORTHO HGT: [No NGS Geoid Model Available.]
                         UTM (Zone 11)         SPC (0404 CA 4)
Northing (Y) [meters]     4024219.772           612842.492
Easting (X)  [meters]      341727.091          2021197.302
US NATIONAL GRID DESIGNATOR: 11SLA4172724220(NAD 83)
          1         2         3         4         5         6         7         8
0123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890
```

#|hide
## Functions.

#|hide
### def decode_OPUS()

In [4]:
#| export
def decode_OPUS(
  txt='',        # A string containing the contents of an entire OPUS report.
  opus_file=''   # A filename of an OPUS Report.
) -> object:      # Decoded class variable of the OPUS report.
  '''
  Decoder for NOAA NGS OPUS reports.
  '''
  if txt:
    opus_txt = txt
  elif opus_file:
    with open(opus_file, 'r') as f:
      opus_txt = f.read()
  else:
    raise Exception('No input provided.')
  class POINT:
    pass
  class OPUS:
    pass
  class STRINGS:
    pass
  opus             = OPUS()
  opus.nad83       = POINT()
  opus.itrf        = POINT()
  opus.state_plane = POINT()
  opus.strings     = STRINGS()
  for row in opus_txt.split('\n'):
    
    ts = row[29:70].strip()
    if ts == 'NGS OPUS SOLUTION REPORT':
      opus.title = ts
    elif ts == 'NGS OPUS-RS SOLUTION REPORT':
      opus.title = ts

    elif row[23:30].strip()       == 'UTM (':
      opus.strings.utm             = row[34:37].strip()
      opus.utm_zone                = int( opus.strings.utm )
      opus.strings.state_plane_key = ts = row[47:51].strip()  # "SPC"
      if ts == "SPC":
        opus.strings.state_plane_id= row[52:62].strip("() ")
        opus.state_plane.id        = opus.strings.state_plane_id

    elif row[0:9].strip()  == 'Northing':
      opus.strings.northing = row[23:37]
      opus.northing         = float( opus.strings.northing )
      if opus.strings.state_plane_key == 'SPC':
        opus.strings.state_plane_northing = ts = row[45:59].strip()
        opus.state_plane.northing = float(opus.strings.state_plane_northing)

    elif row[0:8].strip()  == 'Easting':
      opus.strings.easting  = row[23:37]
      opus.easting          = float( opus.strings.easting )
      if opus.strings.state_plane_key == 'SPC':
        opus.strings.state_plane_easting = ts = row[45:59].strip()
        opus.state_plane.easting = float(opus.strings.state_plane_easting)

    elif row[0:12].strip()    == 'USER:':
      opus.user                = row[11:50].strip()
      opus.strings.run_datestr = ts = row[58:73].strip()
      opus.run_date            = dt.datetime.strptime( ts, "%B %d, %Y")

    elif row[0:12].strip() == 'SOFTWARE:':
      opus.software         = row[12:51].strip()
      opus.strings.start    = ts = row[58:81].strip()
      opus.start            = dt.datetime.strptime(ts, "%Y/%m/%d %H:%M:%S" )

    elif row[0:12].strip() == 'RINEX FILE:':
      opus.rinex_file       = opus.strings.rinex_file = row[11:25].strip()
      opus.strings.run_time = row[58:73].strip()
      opus.run_datetime     = dt.datetime.strptime(opus.strings.run_datestr + " " +
                                                   opus.strings.run_time,
                                                   "%B %d, %Y %H:%M:%S UTC")
      #opus.run_time         = dt.datetime.strptime(ts, "%H:%M:%S" )

    elif row[0:12].strip() == 'EPHEMERIS:':
      opus.ephermis         = row[25:38].strip('[] ')
      opus.strings.stop     = ts = row[58:81].strip()
      opus.stop             = dt.datetime.strptime(ts, "%Y/%m/%d %H:%M:%S" )

    elif row[0:12].strip() == 'ANT NAME:':
      opus.ant_name         = row[11:42].strip()

    elif row[0:12].strip() == 'ARP HEIGHT:':
      opus.strings.arp_height = row[11:17].strip()
      opus.arp_height         = float( opus.strings.arp_height )

    elif row[0:12].strip() == 'REF FRAME:':
      #opus.ref_frame        =  row[11:81].strip()
      opus.strings.nad83_ref_frame = ts = row[11:45].strip()
      opus.nad83.ref_frame  = ts
      opus.strings.itfr_ref_frame  = ts = row[50:81].strip()
      opus.itrf.ref_frame  = ts

    elif row[0:12].strip() ==   'X:':
      opus.strings.nad83_x  = row[15:28]
      opus.nad83.x          = float( opus.strings.nad83_x )
      opus.strings.itrf_x   = row[51:64]
      opus.itrf.x           = float( opus.strings.itrf_x )

    elif row[0:12].strip() ==   'Y:':
      opus.strings.nad83_y  = row[15:28]
      opus.nad83.y          = float( opus.strings.nad83_y )
      opus.strings.itrf_y   = row[51:64]
      opus.itrf.y           = float( opus.strings.itrf_y )

    elif row[0:12].strip() ==   'Z:':
      opus.strings.nad83_z  = row[15:28]
      opus.nad83.z          = float( opus.strings.nad83_z )
      opus.strings.itrf_z   = row[51:64]
      opus.itrf.z           = float( opus.strings.itrf_z )

    elif row[0:12].strip() == 'LAT:':
      st = opus.strings.nad83_lat = row[13:16], row[17:19], row[20:28]
      if float(st[0] ) < 0.0:
        nsf = -1.0
      else:
        nsf = 1.0
      opus.nad83.lat = nsf*np.sum( [abs(float(st[0])), float(st[1])/60, float( st[2])/3600.0])
      st = opus.strings.itrf_lat  = row[49:52], row[53:55], row[56:64]
      opus.itrf.lat = nsf*np.sum( [ abs(float(st[0])), float(st[1])/60, float( st[2])/3600.0])
      ts = opus.strings.lat_rms = row[32:39].strip()
      opus.lat_rms  = float(ts)

    elif row[0:12].strip() == 'W LON:':
      st = opus.strings.nad83_lon = row[13:16], row[17:19], row[20:28]
      opus.nad83.lon = -np.sum( [float(st[0]), float(st[1])/60, float( st[2])/3600.0])
      st = opus.strings.itrf_lon = row[49:52], row[53:55], row[56:64]
      opus.itrf.lon = -np.sum( [float(st[0]), float(st[1])/60, float( st[2])/3600.0])
      ts = opus.strings.lon_rms = row[32:39].strip()
      opus.lon_rms  = float(ts)

    elif row[0:12].strip() == 'EL HGT:':
      st = opus.strings.nad83_el_hgt = row[19:28]
      opus.nad83.el_hgt = float(st)
      st = opus.strings.itrf_el_hgt  = row[55:64]
      opus.itrf.el_hgt = float(st)
      ts = opus.strings.el_rms = row[32:39].strip()
      opus.el_rms  = float(ts)

    elif row[0:12].strip() == 'ORTHO HGT:':
      st = opus.strings.navd88_hgt = row[19:28]
      try:
        opus.nad83.navd88_hgt = float(st)
      except:
        opus.nad83.navd88_hgt = row[12:45].strip()


    elif row[0:28].strip() == 'US NATIONAL GRID DESIGNATOR:':
      opus.us_grid = row[28:44].strip()
  return opus

## Testing

### Test with OPUS data in a string.

In [5]:
def test_decoder( opus_file='', txt = '' ):
  
  if opus_file:
    opus = decode_OPUS( opus_file = opus_file )
    print(f"File: { opus_file }")
  else:
    opus = decode_OPUS( txt = opus_txt )
    
  print(  "==========[ Extracted OPUS Data Strings =======")
  for k in opus.strings.__dict__:
    print(f'{k:24s} = {opus.strings.__dict__[k]}')
  
  print("\n==========[ Extracted Common OPUS Data  ]======")
  for k in opus.__dict__:
    print(f'{k:24s} = {opus.__dict__[k]}')
  
  print("\n==========[ Extracted NAD83 data ]=============")
  for k in opus.nad83.__dict__:
    print(f'{k:24s} = {opus.nad83.__dict__[k]}')
  
  print("\n==========[ Extracted ITRF data ]==============")
  for k in opus.itrf.__dict__:
    print(f'{k:24s} = {opus.itrf.__dict__[k]}')

  print("\n==========[ Extracted State Plane data ]=======")
  for k in opus.state_plane.__dict__:
    print(f'{k:24s} = {opus.state_plane.__dict__[k]}')

  print(f"\n itrfZ - nad83Z: { opus.itrf.el_hgt - opus.nad83.el_hgt : 6.3f}")
  return opus

#### Test with OPSU string.

In [6]:
# Test with the entire OPUS file in a string.
opus = test_decoder( txt = opus_txt )

==========[ Extracted OPUS Data Strings =======
run_datestr              = May 20, 2024
rinex_file               = sfkw121p.24o
run_time                 = 03:53:37 UTC
start                    = 2024/04/30  15:51:00
stop                     = 2024/04/30  22:51:00
arp_height               = 0.000
nad83_ref_frame          = NAD_83(2011)(EPOCH:2010.0000)
itfr_ref_frame           = ITRF2014 (EPOCH:2024.3301)
nad83_x                  =  -2475273.328
itrf_x                   =  -2475274.384
nad83_y                  =  -4509243.734
itrf_y                   =  -4509242.286
nad83_z                  =   3760194.885
itrf_z                   =   3760194.828
nad83_lat                = (' 36', '21', ' 0.18896')
itrf_lat                 = (' 36', '21', ' 0.20210')
lat_rms                  = 0.022
nad83_lon                = ('118', '45', '49.83289')
itrf_lon                 = ('118', '45', '49.89794')
lon_rms                  = 0.012
nad83_el_hgt             =  1094.647
itrf_el_hgt              =  109

#### Test stop - start times.

In [7]:
t = (opus.stop - opus.start)
t.seconds

25200

#### sthl.   Test.

In [8]:
test_decoder( opus_file = '../test_data/sthl-opus.txt' )

File: ../test_data/sthl-opus.txt
==========[ Extracted OPUS Data Strings =======
run_datestr              = May 31, 2024
rinex_file               = sthl061a.24o
run_time                 = 16:13:56 UTC
start                    = 2024/03/01  00:00:00
stop                     = 2024/03/01  03:00:00
arp_height               = 0.000
nad83_ref_frame          = NAD_83(2011)(EPOCH:2010.0000)
itfr_ref_frame           = ITRF2014 (EPOCH:2024.1641)
nad83_x                  =   6104818.188
itrf_x                   =   6104817.320
nad83_y                  =   -605830.498
itrf_y                   =   -605827.720
nad83_z                  =  -1740739.337
itrf_z                   =  -1740738.608
nad83_lat                = ('-15', '56', '33.12668')
itrf_lat                 = ('-15', '56', '33.11405')
lat_rms                  = 0.017
nad83_lon                = ('  5', '40', ' 2.52440')
itrf_lon                 = ('  5', '40', ' 2.43433')
lon_rms                  = 0.026
nad83_el_hgt             =   454.50

<__main__.decode_OPUS.<locals>.OPUS at 0x7f1238200110>

#### OPUS-california, Test.

In [9]:
# Test with an OPUS file.
test_decoder( '../test_data/OPUS-california.txt' )

File: ../test_data/OPUS-california.txt
==========[ Extracted OPUS Data Strings =======
run_datestr              = May 20, 2024
rinex_file               = sfkw121p.24o
run_time                 = 03:53:37 UTC
start                    = 2024/04/30  15:51:00
stop                     = 2024/04/30  22:51:00
arp_height               = 0.000
nad83_ref_frame          = NAD_83(2011)(EPOCH:2010.0000)
itfr_ref_frame           = ITRF2014 (EPOCH:2024.3301)
nad83_x                  =  -2475273.328
itrf_x                   =  -2475274.384
nad83_y                  =  -4509243.734
itrf_y                   =  -4509242.286
nad83_z                  =   3760194.885
itrf_z                   =   3760194.828
nad83_lat                = (' 36', '21', ' 0.18896')
itrf_lat                 = (' 36', '21', ' 0.20210')
lat_rms                  = 0.022
nad83_lon                = ('118', '45', '49.83289')
itrf_lon                 = ('118', '45', '49.89794')
lon_rms                  = 0.012
nad83_el_hgt             =  1

<__main__.decode_OPUS.<locals>.OPUS at 0x7f1238201dd0>

#### zefr, Test.

In [10]:
test_decoder( '../test_data/zefr-opus.txt' )

File: ../test_data/zefr-opus.txt
==========[ Extracted OPUS Data Strings =======
run_datestr              = May 31, 2024
rinex_file               = zefr151a.24o
run_time                 = 14:49:06 UTC
start                    = 2024/05/30  00:00:00
stop                     = 2024/05/30  02:00:00
arp_height               = 0.000
nad83_ref_frame          = NAD_83(2011)(EPOCH:2010.0000)
itfr_ref_frame           = ITRF2014 (EPOCH:2024.4099)
nad83_x                  =    766681.326
itrf_x                   =    766680.442
nad83_y                  =  -5571330.066
itrf_y                   =  -5571328.511
nad83_z                  =   2998751.262
itrf_z                   =   2998751.104
nad83_lat                = (' 28', '13', '39.32263')
itrf_lat                 = (' 28', '13', '39.34362')
lat_rms                  = 0.003
nad83_lon                = (' 82', ' 9', '52.67178')
itrf_lon                 = (' 82', ' 9', '52.69612')
lon_rms                  = 0.015
nad83_el_hgt             =     0.11

<__main__.decode_OPUS.<locals>.OPUS at 0x7f1238202890>

#### zefr-extended, Test.

In [11]:
test_decoder( '../test_data/zefr-opus-extended.txt' )

File: ../test_data/zefr-opus-extended.txt
==========[ Extracted OPUS Data Strings =======
run_datestr              = May 31, 2024
rinex_file               = zefr151a.24o
run_time                 = 14:52:15 UTC
start                    = 2024/05/30 00:00:00
stop                     = 2024/05/30 02:00:00
arp_height               = 0.000
nad83_ref_frame          = NAD_83(2011)(EPOCH:2010.0000)
itfr_ref_frame           = ITRF2014 (EPOCH:2024.40995
nad83_x                  =    766681.329
itrf_x                   =    766680.445
nad83_y                  =  -5571330.069
itrf_y                   =  -5571328.514
nad83_z                  =   2998751.263
itrf_z                   =   2998751.105
nad83_lat                = (' 28', '13', '39.32260')
itrf_lat                 = (' 28', '13', '39.34362')
lat_rms                  = 0.006
nad83_lon                = (' 82', ' 9', '52.67171')
itrf_lon                 = (' 82', ' 9', '52.69604')
lon_rms                  = 0.012
nad83_el_hgt             =  

<__main__.decode_OPUS.<locals>.OPUS at 0x7f12381fb390>

#### ccj2, Test.

In [12]:
test_decoder( '../test_data/ccj2-opus.txt' )

File: ../test_data/ccj2-opus.txt
==========[ Extracted OPUS Data Strings =======
run_datestr              = May 31, 2024
rinex_file               = ccj2061a.24o
run_time                 = 16:48:04 UTC
start                    = 2024/03/01  00:00:00
stop                     = 2024/03/01  03:00:00
arp_height               = 0.000
nad83_ref_frame          = NAD_83(2011)(EPOCH:2010.0000)
itfr_ref_frame           = ITRF2014 (EPOCH:2024.1641)
nad83_x                  =  -4490604.139
itrf_x                   =  -4490604.966
nad83_y                  =   3483893.479
itrf_y                   =   3483895.132
nad83_z                  =   2884927.227
itrf_z                   =   2884928.350
nad83_lat                = (' 27', ' 4', ' 3.11221')
itrf_lat                 = (' 27', ' 4', ' 3.12006')
lat_rms                  = 0.041
nad83_lon                = ('217', '48', '17.88959')
itrf_lon                 = ('217', '48', '17.91857')
lon_rms                  = 0.093
nad83_el_hgt             =   102.17

<__main__.decode_OPUS.<locals>.OPUS at 0x7f12381f89d0>

#### iser, Test

In [13]:
test_decoder('../test_data/iser-opus.txt')

File: ../test_data/iser-opus.txt
==========[ Extracted OPUS Data Strings =======
run_datestr              = May 31, 2024
rinex_file               = iser061a.24o
run_time                 = 17:04:03 UTC
start                    = 2024/03/01  00:00:00
stop                     = 2024/03/01  03:00:00
arp_height               = 0.000
nad83_ref_frame          = NAD_83(2011)(EPOCH:2010.0000)
itfr_ref_frame           = ITRF2014 (EPOCH:2024.1641)
nad83_x                  =   3708162.254
itrf_x                   =   3708160.660
nad83_y                  =   3582296.317
itrf_y                   =   3582298.007
nad83_z                  =   3742780.438
itrf_z                   =   3742781.769
nad83_lat                = (' 36', ' 9', '35.45100')
itrf_lat                 = (' 36', ' 9', '35.48531')
lat_rms                  = 0.005
nad83_lon                = ('315', '59', '20.69977')
itrf_lon                 = ('315', '59', '20.60686')
lon_rms                  = 0.014
nad83_el_hgt             =   430.45

<__main__.decode_OPUS.<locals>.OPUS at 0x7f12381f5e10>

hikb, Test.

In [14]:
test_decoder( '../test_data/hikb-opus.txt' )

File: ../test_data/hikb-opus.txt
==========[ Extracted OPUS Data Strings =======
run_datestr              = May 31, 2024
rinex_file               = hikb061a.24o
run_time                 = 17:05:35 UTC
start                    = 2024/03/01  00:00:00
stop                     = 2024/03/01  03:00:00
arp_height               = 0.000
nad83_ref_frame          = NAD_83(2011)(EPOCH:2010.0000)
itfr_ref_frame           = ITRF2014 (EPOCH:2024.1641)
nad83_x                  =  -5060142.450
itrf_x                   =  -5060143.687
nad83_y                  =    149883.062
itrf_y                   =    149885.063
nad83_z                  =  -3867002.703
itrf_z                   =  -3867001.851
nad83_lat                = ('-37', '33', '39.78155')
itrf_lat                 = ('-37', '33', '39.73403')
lat_rms                  = 0.079
nad83_lon                = ('181', '41', '47.84467')
itrf_lon                 = ('181', '41', '47.92466')
lon_rms                  = 0.023
nad83_el_hgt             =   106.89

<__main__.decode_OPUS.<locals>.OPUS at 0x7f12381f4ed0>

#### prmi, Test.

In [15]:
test_decoder( '../test_data/prmi-opus.txt' )

File: ../test_data/prmi-opus.txt
==========[ Extracted OPUS Data Strings =======
run_datestr              = June 03, 2024
rinex_file               = prmi151f.24o
run_time                 = 04:29:27 UTC
start                    = 2024/05/30  05:00:00
stop                     = 2024/05/30  08:00:00
arp_height               = 0.000
nad83_ref_frame          = NAD_83(2011)(EPOCH:2010.0000)
itfr_ref_frame           = ITRF2014 (EPOCH:2024.4106)
nad83_x                  =   2366882.707
itrf_x                   =   2366882.108
nad83_y                  =  -5588326.057
itrf_y                   =  -5588324.165
nad83_z                  =   1955260.602
itrf_z                   =   1955260.589
nad83_lat                = (' 17', '58', '13.42085')
itrf_lat                 = (' 17', '58', '13.44025')
lat_rms                  = 0.009
nad83_lon                = (' 67', ' 2', '43.33999')
itrf_lon                 = (' 67', ' 2', '43.33366')
lon_rms                  = 0.017
nad83_el_hgt             =   -23.5

<__main__.decode_OPUS.<locals>.OPUS at 0x7f12381f6c90>

#### vikh, Test.

In [16]:
test_decoder( '../test_data/vikh-opus.txt' )

File: ../test_data/vikh-opus.txt
==========[ Extracted OPUS Data Strings =======
run_datestr              = June 03, 2024
rinex_file               = vikh121j.24o
run_time                 = 04:34:55 UTC
start                    = 2024/04/30  09:00:00
stop                     = 2024/05/01  05:00:00
arp_height               = 0.000
nad83_ref_frame          = NAD_83(2011)(EPOCH:2010.0000)
itfr_ref_frame           = ITRF2014 (EPOCH:2024.3300)
nad83_x                  =   2587869.448
itrf_x                   =   2587868.842
nad83_y                  =  -5499035.710
itrf_y                   =  -5499033.794
nad83_z                  =   1928483.741
itrf_z                   =   1928483.756
nad83_lat                = (' 17', '42', '58.24483')
itrf_lat                 = (' 17', '42', '58.26499')
lat_rms                  = 0.002
nad83_lon                = (' 64', '47', '53.25446')
itrf_lon                 = (' 64', '47', '53.24536')
lon_rms                  = 0.011
nad83_el_hgt             =    -4.3

<__main__.decode_OPUS.<locals>.OPUS at 0x7f12381b2e90>

#### vith, Test.

In [17]:
test_decoder( '../test_data/vith-opus.txt' )

File: ../test_data/vith-opus.txt
==========[ Extracted OPUS Data Strings =======
run_datestr              = June 03, 2024
rinex_file               = vith101j.24o
run_time                 = 04:35:50 UTC
start                    = 2024/04/10  09:00:00
stop                     = 2024/04/11  05:00:00
arp_height               = 0.000
nad83_ref_frame          = NAD_83(2011)(EPOCH:2010.0000)
itfr_ref_frame           = ITRF2014 (EPOCH:2024.2754)
nad83_x                  =   2562351.844
itrf_x                   =   2562351.231
nad83_y                  =  -5487281.701
itrf_y                   =  -5487279.796
nad83_z                  =   1994491.511
itrf_z                   =   1994491.525
nad83_lat                = (' 18', '20', '35.97850')
itrf_lat                 = (' 18', '20', '35.99923')
lat_rms                  = 0.006
nad83_lon                = (' 64', '58', ' 9.17213')
itrf_lon                 = (' 64', '58', ' 9.16360')
lon_rms                  = 0.002
nad83_el_hgt             =     6.4

<__main__.decode_OPUS.<locals>.OPUS at 0x7f12381b3910>